In [1]:
cd ../

/home/amir/study/langchain_vectordb


/home/amir/.cache/pypoetry/virtualenvs/langchain-vectordb-Chhx0xS9-py3.10/lib/python3.10/site-packages/IPython/core/magics/osm.py:417: UserWarning: This is now an optional IPython functionality, setting dhist requires you to install the `pickleshare` library.
  self.shell.db['dhist'] = compress_dhist(dhist)[-100:]


In [2]:
import os
from dotenv import load_dotenv

load_dotenv()

api_key: str = os.getenv("api_key", "")
base_url: str = os.getenv("base_url", "")

In [3]:
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(
    model="gpt-5-nano-2025-08-07",
    api_key=api_key,
    base_url=base_url,
)


In [4]:
from langchain.prompts import ChatPromptTemplate

prompt_template = ChatPromptTemplate([
    ("system", "Ты полезный AI-ассистент с чувством юмора."),
    ("user", "Расскажи мне шутку на тему - {topic}"),
])
prompt_template.invoke({"topic": "космос"})

ChatPromptValue(messages=[SystemMessage(content='Ты полезный AI-ассистент с чувством юмора.', additional_kwargs={}, response_metadata={}), HumanMessage(content='Расскажи мне шутку на тему - космос', additional_kwargs={}, response_metadata={})])

In [5]:
from langchain.prompts import PromptTemplate

template = """
Перепиши этот текcт в заданном стиле: {input_text}
Стиль: {style}.
Результат:
"""
prompt = PromptTemplate(
    input_variables=['text', 'style'],
    template=template 
)
style_changer_chain = prompt | llm

In [9]:
text = '''Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей, 
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.'''
style = 'Роман 18 века'

answer = style_changer_chain.invoke({'input_text': text, 'style': style})
print(answer)

content='Приветствую вас, досточтимые читатели, на курсе по тюнингу языковых моделей. Здесь мы учимся придать моделям речевым ещё более благородный облик и раскрыть до предела их скрытый потенциал; предлагаем советы по самым свежим методам обучения и настройки этих умных машин, и воспитаем вместе практические умения, необходимые для решения самых сложных задач в деле обработки естественного языка.' additional_kwargs={'refusal': None} response_metadata={'token_usage': {'completion_tokens': 2153, 'prompt_tokens': 113, 'total_tokens': 2266, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 2048, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-CLtfgysqocamPKMTt7RCQDgMA5YEN', 'service_tier': 'default', 'finish_reason': 'stop', 'logprobs': None} id='run--080c4f0d-accd-4d73-87ad-512cbd3c211f-0' usage_metadat

In [10]:
print(answer.content)

Приветствую вас, досточтимые читатели, на курсе по тюнингу языковых моделей. Здесь мы учимся придать моделям речевым ещё более благородный облик и раскрыть до предела их скрытый потенциал; предлагаем советы по самым свежим методам обучения и настройки этих умных машин, и воспитаем вместе практические умения, необходимые для решения самых сложных задач в деле обработки естественного языка.


In [11]:
batch_answer = style_changer_chain.batch([
    {'input_text': text, 'style': 'В стиле времен советского союза 1950 гг'},
    {'input_text': text, 'style': 'В стиле поэмы'}
])

In [13]:
batch_answer[0].content

'Товарищи! Добро пожаловать на курс по тюнингу языковых моделей. Здесь мы учимся превращать модели в ещё более мощные и крепкие инструменты, раскрывая их потенциал до предела возможностей науки и техники. Мы дадим вам советы по передовым методам обучения и настройки языковых моделей, а также обучим практическим навыкам для решения самых сложных задач в области обработки естественного языка. Да здравствует научный труд, да здравствует прогресс и победы техники!'

In [15]:
print(batch_answer[1].content)

Добро пожаловать на курс по тюнингу языковых моделей,
где слова и коды сплетаются в новый гимн данных.
Мы учим вас делать модели ещё круче,
раскрывать их потенциал до максимума — без границ.

Мы дадим советы по самым свежим методам обучения
и тонкой настройке языковых моделей,
а также научим практическим навыкам для решения
сложнейших задач в обработке естественного языка.

Пусть этот путь звучит стихами алгоритмов,
пусть каждая идея загорается искрой новых трактовок,
и ваш опыт растёт — шаг за шагом к вершинам понимания.


In [16]:
from langchain.schema.output_parser import StrOutputParser
chain_with_parser = prompt | llm | StrOutputParser()

print(chain_with_parser.invoke({'input_text': text, 'style': 'В стиле поэмы'}))

Добро пожаловать в курс тюнинга языковых моделей,
где шепчет нейронный океан и искрят строки данных.
Мы учим: как сделать модели ещё круче,
как раскрыть их потенциал до максимума — без границ и стен.

Мы принесём советы по самым свежим методам обучения и настройки,
как ветер—между слоями и параметрами лёгко проскользнуть.
И научим практическим навыкам для решения задач,
что в обработке естественного языка стоят над веками вопросов.


In [17]:
import re

def del_spaces(inputs: dict) -> dict:
    text = inputs["input_text"]
    style = inputs["style"]
    
    # заменяем пустые строки и дополнительные пробелы на один, используя регулярные выражения
    text = re.sub(r'(\r\n|\r|\n){2,}', r'\n', text)
    text = re.sub(r'[ \t]+', ' ', text)

    return {"input_text": text, 'style': style}

dirty_text = '''Приветствуем на курсе по        тюнингу языковых        моделей! 

Тут мы учим, как сделать модельки еще круче и раскрыть     их потенциал до максимума. 
Мы дадим вам советы по   самым свежим методам  обучения и настройки языковых                    моделей,

а также научим практическим навыкам для        решения сложных задач в обработке естественного языка.'''

print(del_spaces({'input_text': dirty_text, 'style': 'Рэп'})['input_text'])

Приветствуем на курсе по тюнингу языковых моделей! 
Тут мы учим, как сделать модельки еще круче и раскрыть их потенциал до максимума. 
Мы дадим вам советы по самым свежим методам обучения и настройки языковых моделей,
а также научим практическим навыкам для решения сложных задач в обработке естественного языка.


In [18]:
seq_chain = del_spaces | prompt | llm |  StrOutputParser() # Готово!!!
print(seq_chain.invoke({'input_text': dirty_text, 'style': 'Рэп'}))

Результат:
Йо, привет на курсе тюнинга языковых моделей,
Здесь поднимаем их крутизну и раскрываем потенциал до максимума.
Мы учим, как сделать модельки ещё круче и зажечь их внутри,
Прокачаем гиперпараметры и методы обучения — без пауз и суеты внутри.
Дадим вам свежайшие советы по настройке и архитектуре,
И на практике научим решать НЛП-задачи — чётко, быстро, без мути и фигуры.
Код на битах, пайплайны в ритме — движемся вперёд без промаха,
Этот рэп-курс — место, где наука встречается с стилем и движухой.


In [19]:
from operator import itemgetter
from langchain.prompts import ChatPromptTemplate

prompt_1 = ChatPromptTemplate.from_template("В каком городе родился {person}?")
prompt_2 = ChatPromptTemplate.from_template("В какой стране находится город {city}? Ответь на {language} языке.")

chain_1 = prompt_1 | llm | StrOutputParser()
chain_2 = (
    {"city": chain_1, "language": itemgetter("language")}
    | prompt_2 
    | llm 
    | StrOutputParser()
)
result = chain_2.invoke({"person": "Колумб", "language": "испанском"})
print(result)

Se encuentra en Italia.


In [21]:
from typing import Literal
from langchain.schema.runnable import RunnableBranch, RunnablePassthrough
from langchain.output_parsers.openai_functions import PydanticAttrOutputFunctionsParser
from pydantic import BaseModel
from langchain_core.utils.function_calling import convert_to_openai_function

In [23]:
botanist_template = """Ты очень опытный флорист и ботаник, знаешь всё о цветах, растениях.
Тебе нравится отвечать на вопросы о том, как выбирать и ухаживать за растениями. 
Ты отвечаешь так, что всё становится ясно даже начинающему цветоводу. 
Вот вопрос:
{input}"""

football_template = """Ты спортивный журналист с большим опытом, твоя основная специализация футбол.
Ты знаешь всё о футбольных командах и игроках, и очень любишь отвечать на вопросы о футболе, но кратко и по делу.
Вот вопрос:
{input}"""

botanist_prompt = PromptTemplate.from_template(botanist_template)
football_prompt = PromptTemplate.from_template(football_template)

# Создаём ветки
prompt_branch = RunnableBranch(
    (lambda x: x["topic"] == "botany", botanist_prompt),
    (lambda x: x["topic"] == "football", football_prompt),
    PromptTemplate.from_template("Answer the question: {input}"),
)

In [26]:
class TopicClassifier(BaseModel):
    "Classify the topic of the user question"
    topic: Literal["botany", "football", "general"]
    
    
classifier_function = convert_to_openai_function(TopicClassifier)

model = ChatOpenAI(
    api_key=api_key,
    base_url=base_url,
    model='gpt-5-nano-2025-08-07', 
    ).bind(
        functions=[classifier_function],
        function_call={"name": "TopicClassifier"}
    )
parser = PydanticAttrOutputFunctionsParser(pydantic_schema=TopicClassifier, attr_name="topic")

classifier_chain = model | parser